In [3]:
import os
from dotenv import load_dotenv
from mistralai import Mistral

load_dotenv()
key_api = os.getenv("MISTRAL_API_KEY")
client = Mistral(api_key=key_api)


In [22]:
from sentence_transformers import SentenceTransformer, util

embedder = SentenceTransformer("all-MiniLM-L6-v2")


Loading weights: 100%|██████████| 103/103 [00:00<00:00, 1523.53it/s, Materializing param=pooler.dense.weight]                             
BertModel LOAD REPORT from: sentence-transformers/all-MiniLM-L6-v2
Key                     | Status     |  | 
------------------------+------------+--+-
embeddings.position_ids | UNEXPECTED |  | 

Notes:
- UNEXPECTED	:can be ignored when loading from different task/architecture; not ok if you expect identical arch.


In [23]:
def generate_variations(question, n=3):
    answers = []
    
    for k in range(n):
        response = client.chat.complete(
            model="mistral-small-latest",
            messages=[{
                "role": "user", "content": question
            }],
            temperature=0.9
        )
        answer = response.choices[0].message.content
        answers.append(answer)
        
    return answers

In [24]:
generate_variations("Qui est le president de Paris?")

['Paris n\'est pas une entité politique indépendante, donc il n\'y a pas de "président de Paris". Cependant, la ville est dirigée par un **maire**, qui est actuellement **Anne Hidalgo** (depuis 2014). Elle est membre du Parti socialiste (PS) et a été réélue en 2020.\n\nSi vous parlez de la France en général, le président actuel est **Emmanuel Macron** (depuis 2017, réélu en 2022).\n\nVous cherchez des informations sur un autre aspect de Paris ou de la France ?',
 "Paris est une ville, et non un pays, donc elle n'a pas de président. Cependant, elle a un maire, qui est actuellement **Anne Hidalgo**, élue depuis 2014 et réélue en 2020.\n\nSi vous parlez de la **Région Île-de-France**, son exécutif est dirigé par un **président du Conseil régional**, qui est actuellement **Valérie Pécresse** (depuis 2015).\n\nN'hésitez pas à préciser votre question si vous cherchez une autre information !",
 "Paris n'est pas une entité politique indépendante avec son propre président. C'est la capitale de 

In [26]:
def calculate_consistency(answers):
    embeddings = embedder.encode(answers, convert_to_numpy=True)
    scores = util.cos_sim(embeddings, embeddings)
    score = (scores[0, 1] + scores[0, 2] + scores[1, 2])/3
    return float(score)
    
    

In [29]:
QUESTION_TEST = "Qui sera elu president des USA en 2028 ?"

variations = generate_variations(QUESTION_TEST)

consistency_score = calculate_consistency(variations)

print(consistency_score)
    

0.8604138493537903
